# Practico 5 (parte 4)

## Entrenar word embeddings

## Importación de módulos y librerías

In [1]:
# Inclusion de librerias y módulos
import os
import logging
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Usamos las stopwords definidas en nltk más algunas propias
from nltk.corpus import stopwords
stopwords = stopwords.words('english') + [',', "’", '.', ':', '-', ';']

# Algunas utilidades
from utiles import print_some_info
from utiles import convert_emojis
from utiles import convert_emoticons
from utiles import bcolors

# Nos permite convertir str a list
from ast import literal_eval

# Importamos wrod2vec de la lib gensim
from gensim.models import Word2Vec

# Importamos logger para tener informacion de estado
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Colores
BLUE   = '#5DADE2'
RED    = '#ff7043'
ORANGE = '#F5B041'
GREEN  = '#58D68D'
YELLOW = '#F4D03F'
pltcolors = [BLUE, RED, ORANGE, GREEN, YELLOW]

# Plot axes y legends parambs
plt.rcParams["axes.labelweight"]   = "bold"
plt.rcParams["axes.titleweight"]   = "bold"
plt.rcParams["legend.shadow"]      = True
plt.rcParams["figure.titleweight"] = "bold"

data_dir = os.path.join('..', 'dataset')

########################################################
filename = 'yup_messages_preprocessed.csv'
# filename = 'dev_yup_messages_preprocessed.csv'

SAVE_CURATED_DATASET = False

## Lectura del archivo de mensajes
Utilizamos unicamente el archivo de mensajes dado que vamos a entrenar un word embeding como word2vec. Entendemos que para el propósito del análisis y por que no estamos empleando ningún modelo de clasificación o regresión podemos usar el conjunto de datos completo.

In [2]:
if SAVE_CURATED_DATASET:
    df = pd.read_csv(os.path.join(data_dir, filename))

    print(f'El conjunto de datos utilizado es {filename}')
    print_some_info(df)
else:
    print('Curación evitada')

El conjunto de datos utilizado es yup_messages_preprocessed.csv
El conjunto de datos posee 1574069 filas y 6 columnas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1574069 entries, 0 to 1574068
Data columns (total 6 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   session_id    1574069 non-null  int64 
 1   created_at    1574069 non-null  object
 2   sent_from     1574069 non-null  object
 3   sent_to       1574069 non-null  object
 4   content_type  1574069 non-null  object
 5   text          1574069 non-null  object
dtypes: int64(1), object(5)
memory usage: 72.1+ MB
None



## Curacion del dataset
Al momento de curar el datset llevamos a cabo los siguientes pasos:

1. Consideramos para el análisis solo las columnas `session_id`, `sent_from` y `text`, en donde la última mencionada contiene los vectores de tokens correspondiente a los turno de cada dialo entre estudiante y tutor.
2. Se remueven las filas que contienen mensajes del sistema o no corresponden con turnos de tutor o estudiante.
3. Se realiza la conversión de tipo del campo texto a lista de strings
4. Se sustituye el caracter unicode correspondiente a un emoji por un token del tipo `:token_emoji:`
5. Incialmente se consideró la lógica para sustituir la cadena de caracteres correspondiente a un emoticon por un token del tipo `:token_emoticon:`, sin embargo y por el momento, no lo tratamos por que requiere de un mejor lógica dado que los paréntesis y signos empleados se confunden con los utlizados en ecuaciones y texto regular. Lo que trae aparejado un elevado tiempo de procesamiento.
6. Se convierte a minúsculas para unificar los casos con mayusculas. De este modo Token sera convertido a token.
7. Como punto de partida consideramos la lista de stop words incluidas en el módulo python NLTK con el agregado de los siguientes signos de puntuación [',', "’", '.', ':', '-', ';'] con el objetivo de reducir el  vocabulario a las palabras de mayor utilidad.
8. Se guarda en un archivo .csv el conjunto de datos curado para el presente caso de análisis.

9. Alternativa para cargar el conjunto de datos ya curado para evitar el tiempo de procesamiento del conjunto de datos crudos.
10. Se repite paso 3 a partir del archivo de datos curados.

In [2]:
fn = os.path.join(data_dir, filename.replace('.csv','_curated.csv'))
if SAVE_CURATED_DATASET:
    #1. Tomamos solo las columnas que nos pueden servir. Esto es preliminar, podríamos tomar solo `text`
    dfclean = df[['session_id', 'sent_from', 'text']]

    #2. Tomamos solo las filas que sean tutor o student a partir de la columna `sent_from`
    dfclean = dfclean[dfclean.sent_from.isin(['student', 'tutor'])]

    #3. Convertimos a lista de strings el contenido de la columna text
    dfclean['text'] = dfclean.text.apply(lambda x: literal_eval(x))

    #4. Se sustituyen emojis por tokens 
    dfclean['text'] = dfclean.text.apply(lambda x: [convert_emojis(w) for w in x])

    #5. Se sustituyen emoticones por palabras 
    # dfclean['text'] = dfclean.text.apply(lambda x: [convert_emoticons(w) for w in x])

    #6. Convernitimos a minúsculas para unificar el tratamiento
    dfclean['text'] = dfclean.text.apply(lambda x: [w.lower() for w in x])

    #7. Removemos las stopwords
    dfclean['text'] = dfclean.text.apply(lambda x: [w for w in x if w not in stopwords])
    
    #8. Se guarda el dataset curado
    dfclean.to_csv(fn, index=False)
else:
    #9. Se carga el dataset curado 
    dfclean = pd.read_csv(fn)
    
    #10. Convertimos a lista de strings el contenido de la columna text
    dfclean['text'] = dfclean.text.apply(lambda x: literal_eval(x))

print_some_info(dfclean)

El conjunto de datos posee 1411477 filas y 3 columnas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1411477 entries, 0 to 1411476
Data columns (total 3 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   session_id  1411477 non-null  int64 
 1   sent_from   1411477 non-null  object
 2   text        1411477 non-null  object
dtypes: int64(1), object(2)
memory usage: 32.3+ MB
None



In [3]:
size_vect = [100, 1000]
sg_vect = [0, 1]
model_vect = list()
fnmodel_vect = list()

for sg in sg_vect:
    for size in size_vect:
        # size = 100
        window = 5
        min_count = 100
        # sg = 0

        params = f'{size}-{window}-{min_count}-{sg}'
        fnmodel = f'{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}_model_{params}.bin'

        model = Word2Vec(list(dfclean.text), size=size, window=window, min_count=min_count, sg=sg, compute_loss=True, workers=4)
        model.save(fnmodel)
        
        model_vect.append(model)
        fnmodel_vect.append(fnmodel)

2020-10-07 00:31:12,185 : INFO : collecting all words and their counts
2020-10-07 00:31:12,186 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-10-07 00:31:12,208 : INFO : PROGRESS: at sentence #10000, processed 41794 words, keeping 3398 word types
2020-10-07 00:31:12,229 : INFO : PROGRESS: at sentence #20000, processed 84698 words, keeping 5244 word types
2020-10-07 00:31:12,254 : INFO : PROGRESS: at sentence #30000, processed 128313 words, keeping 6816 word types
2020-10-07 00:31:12,277 : INFO : PROGRESS: at sentence #40000, processed 172083 words, keeping 8034 word types
2020-10-07 00:31:12,301 : INFO : PROGRESS: at sentence #50000, processed 214137 words, keeping 9180 word types
2020-10-07 00:31:12,321 : INFO : PROGRESS: at sentence #60000, processed 257601 words, keeping 10338 word types
2020-10-07 00:31:12,354 : INFO : PROGRESS: at sentence #70000, processed 300247 words, keeping 11506 word types
2020-10-07 00:31:12,377 : INFO : PROGRESS: at sentenc

In [9]:
for i, model in enumerate(model_vect):
    print(f'{bcolors.HEADER}{fnmodel_vect[i]}{bcolors.ENDC}')
    print(f'{bcolors.OKGREEN}Primeras 50 compoentes del vocabulario{bcolors.ENDC}')
    print(f'{bcolors.OKBLUE}{list(model.wv.vocab.keys())[0:50]}{bcolors.ENDC}')
    print(f'{bcolors.OKGREEN}Ultimas 50 compoentes del vocabulario{bcolors.ENDC}')
    print(f'{bcolors.OKBLUE}{list(model.wv.vocab.keys())[-50:]}{bcolors.ENDC}')
    print(f'{bcolors.FAIL}#######################################{bcolors.ENDC}')

20201007-003111_model_100-5-100-0.bin
Primeras 50 compoentes del vocabulario
['<url>', 'hey', '!', 'welcome', 'yup', '', 'looking', 'problem', "'ve", 'reviewed', 'finding', 'domain', 'square', 'root', 'function', 'let', "'s", 'work', 'together', 'find', 'exactly', "'re", 'stuck', 'tried', '?', 'simplifying', 'wrong', 'okay', 'actually', 'need', 'simplify', 'however', 'would', "n't", 'please', 'show', 'check', 'appreciate', 'seem', 'discuss', 'move', 'tell', 'mean', 'word', '"', 'meant', 'learned', 'far', 'x', 'good']
Ultimas 50 compoentes del vocabulario
['opens', 'neutrons', 'varma', 'pages', 'x/2', '):', 'electron', 'h+', 'ff', 'collinear', 'eliza', 'terminal', 'amber', 'bonds', 'ticket', 'tickets', 'bonding', 'delta', 'natalie', 'joe', 'breanna', 'f(1', 'molecule', 'pizza', 'matrices', 'oscar', 'wavelength', 'valence', 'slices', 'mackenzie', 'definite', 'yeap', 'index', 'riley', 'dipole', 'shinde', 'candaza', 'alonzo', 'okaye', 'dora', 'vavilakolanu', 'fn', 'kothari', ':-d', 'krysta

In [26]:
for i, model in enumerate(model_vect):
    print(f'{bcolors.HEADER}{fnmodel_vect[i]}{bcolors.ENDC}')
    for ms in model.wv.most_similar("good"):
        print(f'{bcolors.OKBLUE}{ms}{bcolors.ENDC}')
    print(f'{bcolors.FAIL}#######################################{bcolors.ENDC}')

20201007-003111_model_100-5-100-0.bin
('great', 0.7975860238075256)
('excellent', 0.6769210696220398)
('nice', 0.6664376258850098)
('awesome', 0.5815260410308838)
('amazing', 0.5421160459518433)
('fantastic', 0.5264471769332886)
('wonderful', 0.5258111953735352)
('perfect', 0.46715396642684937)
('nailed', 0.43611302971839905)
('brilliant', 0.4184577465057373)
#######################################
20201007-003142_model_1000-5-100-0.bin
('great', 0.7545366287231445)
('excellent', 0.6358898878097534)
('nice', 0.6242243051528931)
('awesome', 0.5317827463150024)
('amazing', 0.5181410312652588)
('wonderful', 0.5166220664978027)
('fantastic', 0.4775938391685486)
('nailed', 0.4380299746990204)
('brilliant', 0.42936575412750244)
('perfect', 0.4292442798614502)
#######################################
20201007-003248_model_100-5-100-1.bin
('great', 0.6996840238571167)
('lovely', 0.6376404166221619)
('amazing', 0.6048358678817749)
('nice', 0.5867875814437866)
('mhm', 0.5790700316429138)
('excell